# Trabajo en Case 1

In [33]:
import numpy as np


In [34]:
# Teoría
# 1. Indique cual es costo de "Training" y de test del algoritmo KNN.
# R/ El costo de entrenar es O(1), ya que es solo la carga de datos. El costo de entrenamiento es de O(N*D), donde N 
# es la cantidad de datos, mientras que D son los features. En el peor caso donde N = D, este tendría un costo de O(N^2)

# 2. Explique por qué se dice que instance learning es un algoritmo por fuerza bruta o perezoso.
# R/ Este modelo solo se basa en comparación del dato a etiquetar con los N datos cargados en memoria, no existe
# un modelo que haga una simplificación, sólo comparaciones para aproximar el dato.

# 3. Indique cuántas comparaciones se deben realizar en la etapa de test (asumiendo el peor de los casos).
# R/ N*D pruebas, donde N es la cantidad de datos, mientras que D son los features. En el peor caso donde N = D, 
# este tendria que hacer N^2 comparaciones

# 4. Investigue alguna modificación que se ha realizado al algoritmo original del KNN.
# R/ Una variante del algoritmo es el árbol KD, el cual consiste en mejorar la eficiencia de encontrar los k-vecinos 
# más cercanos al indexar los ejemplos de entrenamiento en una estructura de árbol binario, donde en cada nivel del árbol el
# algoritmo selecciona una característica y divide los ejemplos de entrenamiento en dos subárboles según el valor medio de 
# esa característica.


In [52]:
class KNearestNeighbors():
    def __init__(self, X_train, y_train, n_neighbors=5, weights='uniform'):

        self.X_train = X_train
        self.y_train = y_train

        self.n_neighbors = n_neighbors
        self.weights = weights

        self.n_classes = 3

    def euclidian_distance(self, a, b):
        return np.sqrt(np.sum((a - b)**2, axis=1))

    def manhattan_distance(self, a, b):
        return np.abs(np.sum((a - b)**2, axis=1))

    def kneighbors(self, X_test, return_distance=False):

        dist = []
        neigh_ind = []
        point_dist = []
        
        distance_function = self.euclidian_distance if self.weights == 'uniform' else self.manhattan_distance
        
        for x_test in X_test:
            point_dist += [ distance_function(x_test, self.X_train) ]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh,
                                  key=lambda x: x[1])[:self.n_neighbors]

            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]

            dist.append(dist_list)
            neigh_ind.append(ind_list)

        if return_distance:
            return np.array(dist), np.array(neigh_ind)

        return np.array(neigh_ind)

    def predict(self, X_test):
        
        neighbors = self.kneighbors(X_test)
        y_pred = np.array([
            np.argmax(np.bincount(self.y_train[neighbor]))
            for neighbor in neighbors
        ])
        return y_pred

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return float(sum(y_pred == y_test)) / float(len(y_test)), y_pred, y_test


# Iris dataset (Observations/Treatments)

In [53]:
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from matplotlib import pyplot as plt
dataset = datasets.load_iris(as_frame=True)

X = dataset.data
y = dataset.target


In [54]:
mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

In [62]:
if isinstance(X, pd.DataFrame):
    X = X.to_numpy()
    y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(\
                X, y, test_size=0.3, random_state=12)



our_classifier = KNearestNeighbors(X_train, y_train, n_neighbors=50, weights='distance')
sklearn_classifier = KNeighborsClassifier(n_neighbors=50).fit(X_train, y_train)

our_accuracy, y_pred, y_test = our_classifier.score(X_test, y_test)
sklearn_accuracy = sklearn_classifier.score(X_test, y_test)

print(y_pred[:20], y_test[:20])

pd.DataFrame([[our_accuracy, sklearn_accuracy]],
             ['Accuracy'],    
             ['Our Implementation', 'Sklearn\'s Implementation'])

[0 1 0 1 2 2 2 0 2 0 1 0 0 0 2 2 2 1 0 1] [0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 1]


,Our Implementation,Sklearn's Implementation
Accuracy,0.888889,0.888889
